In [41]:
# !pip install tensorflow keras numpy matplotlib seaborn tqdm scikit-learn

In [1]:
from utils import *

import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from datetime import datetime
from datetime import timedelta
from tqdm import tqdm

sns.set()

# Configure Modeling Parameters and Fetch Data

Enter a ticker and date range you would like to build the model on.  This model takes a a single ticker's data.  Also enter a training size for the proportion of the data you want to include in your training set vs. your test set.

In [2]:
# Configurations

# stock configs
ticker = ['TSLA']
start_date = '2020-01-01'
end_date = '2024-05-05'

# model configs
train_size = 0.8

In [5]:
# Data Fetching
data = fetch_stock_data(ticker, start_date, end_date)[ticker[0]]
print(data.shape)
included_days = len(data)
data.head()

(1092, 8)


,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits
0,2020-01-02 00:00:00-05:00,28.299999,28.713333,28.114000,28.684000,142981500,0.0,0.0
1,2020-01-03 00:00:00-05:00,29.366667,30.266666,29.128000,29.534000,266677500,0.0,0.0
2,2020-01-06 00:00:00-05:00,29.364668,30.104000,29.333332,30.102667,151995000,0.0,0.0
3,2020-01-07 00:00:00-05:00,30.760000,31.441999,30.224001,31.270666,268231500,0.0,0.0
4,2020-01-08 00:00:00-05:00,31.580000,33.232666,31.215334,32.809334,467164500,0.0,0.0


In [30]:
# Normalize the 'Close' price column using MinMaxScaler
minmax = MinMaxScaler().fit(data.iloc[:, 4:5].astype('float32')) # Close index
df_log = minmax.transform(data.iloc[:, 4:5].astype('float32')) # Close index
df_log = pd.DataFrame(df_log)
df_log.head()

,0
0,0.128385
1,0.268205
2,0.138573
3,0.269962
4,0.494826


# Model Implementation

Below we implement the model, first we separate the data into a train-test split.  For this LSTM model, we will leverage continuous windows of time.

In [32]:
# Define the sizes for testing and simulation
test_size = included_days * (1 - train_size)
simulation_size = 10

# Split the data into training and testing sets
df_train = df_log.iloc[:-test_size]
df_test = df_log.iloc[-test_size:]
data.shape, df_train.shape, df_test.shape

TypeError: cannot do positional indexing on RangeIndex with these indexers [-218.39999999999995] of type float